In [1]:
import tensorrt as trt
import pycuda.driver as cuda
import pycuda.autoinit
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist
import time

In [2]:
# Helper function to load a TensorRT engine
def load_trt_engine(engine_path):
    with open(engine_path, 'rb') as f:
        runtime = trt.Runtime(trt.Logger(trt.Logger.WARNING))
        engine = runtime.deserialize_cuda_engine(f.read())
    return engine

In [4]:
# Prepare the TensorRT execution context
def create_context(engine):
    context = engine.create_execution_context()
    return context

In [5]:
# Load the TensorRT model
trt_engine_path = r'O:\Training Assignments\Resnet\TensorRT\baseResnet.trt' # Replace with your TRT model path
engine = load_trt_engine(trt_engine_path)
context = create_context(engine)


In [6]:
# Load MNIST data
(_, _), (x_test, y_test) = mnist.load_data()

# Normalize and reshape the image to (28, 28, 1)
x_test = x_test / 255.0
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

In [7]:

# Pick a random image
random_index = np.random.randint(0, x_test.shape[0])
random_image = x_test[random_index]
true_label = y_test[random_index]

# Prepare buffers
input_shape = (1, 28, 28, 1)
output_shape = (1, 10)  # Assuming 10 classes for MNIST

Prediction latency: 0.003658 seconds
The prediction is correct! Predicted: 3


In [8]:
input_data = np.expand_dims(random_image, axis=0).astype(np.float32)

# Allocate buffers for inputs and outputs
d_input = cuda.mem_alloc(input_data.nbytes)
d_output = cuda.mem_alloc(np.empty(output_shape, dtype=np.float32).nbytes)
bindings = [int(d_input), int(d_output)]

# Copy input to device
cuda.memcpy_htod(d_input, input_data)

# Measure the time before prediction
start_time = time.time()

# Run inference
context.execute_v2(bindings)

# Measure the time after prediction
end_time = time.time()

# Copy output from device
output_data = np.empty(output_shape, dtype=np.float32)
cuda.memcpy_dtoh(output_data, d_output)

# Calculate the latency
latency = end_time - start_time

# Extract the predicted label
predicted_label = np.argmax(output_data)

# Plot the image
plt.imshow(random_image.squeeze(), cmap='gray')
plt.title(f"True label: {true_label}, Predicted label: {predicted_label}")
plt.show()

# Print the latency
print(f"Prediction latency: {latency:.6f} seconds")

# Print whether the prediction is correct or not
if true_label == predicted_label:
    print(f"The prediction is correct! Predicted: {predicted_label}")
else:
    print(f"The prediction is incorrect. Predicted: {predicted_label}, True label: {true_label}")